Test if data generator works

In [1]:
import numpy as np
import torch
torch.set_default_dtype(torch.float64)
import random_generators #local

# from scipy.stats import uniform_direction
# import Klein_Bottle.kb_math as kb_math

from tqdm import tqdm

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
n_steps = 10 # Sequence Length
N = 10 # Batch Size
max_hop = 0.1 # Scale parameter for velocities



# Generate Initial Points
X0 = 2 * torch.pi * torch.rand(N,2)

V, pos = torch.zeros((N, n_steps, 2)), torch.zeros((N, n_steps, 2))

minibatch_size = 10
for i in tqdm(range(N // minibatch_size)):
    V[minibatch_size*i:minibatch_size*(i+1)], pos[minibatch_size*i:minibatch_size*(i+1)] = random_generators.random_trajectories(X0=X0[minibatch_size*i:minibatch_size*(i+1)], n_steps=n_steps, max_hop=max_hop)

100%|██████████| 1/1 [00:17<00:00, 17.79s/it]


In [2]:
X0.shape, V.shape, pos.shape

(torch.Size([10, 2]), torch.Size([10, 10, 2]), torch.Size([10, 10, 2]))

In [1]:
import torch
import torch.nn as nn
class RNN_multilayer(nn.Module):
    def __init__(self, hidden_size, input_size=2, output_size=2, num_layers=2):
        super(RNN_multilayer, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # self.encoder = nn.Linear(2*output_size, hidden_size)
        # self.encoder = [nn.Linear(input_size, hidden_size)]*num_layers # original version

        self.encoder = nn.ModuleList([nn.Linear(input_size, hidden_size) for _ in range(num_layers)])
        self.rnn = nn.RNN(input_size, hidden_size, num_layers,nonlinearity='relu', bias=False, batch_first=True)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x_0, V):
        # Initialize hidden state
        h_0 = torch.stack([self.encoder[i](x_0) for i in range(self.num_layers)], dim=0)

        # Forward pass through RNN
        out = self.rnn(V, h_0)
        # out = self.decoder(out[1][-1])
        out = self.decoder(out[0])

        return out

model = RNN_multilayer(hidden_size=8)
model.state_dict().keys()

odict_keys(['encoder.0.weight', 'encoder.0.bias', 'encoder.1.weight', 'encoder.1.bias', 'rnn.weight_ih_l0', 'rnn.weight_hh_l0', 'rnn.weight_ih_l1', 'rnn.weight_hh_l1', 'decoder.weight', 'decoder.bias'])